# Exploration of a bulk download from ETER

[ETER](https://www.eter-project.com/) is the European Tertiary Education Register. It contains institution-level information about skills supply in the EU. We have bulk-downloaded the data and here we check some of its characteristics.

In [ ]:
%run ../notebook_preamble.ipy

import seaborn as sn

from eis.utils.data_processing import *

## Read data

In [ ]:
eter = pd.read_csv(f'{project_dir}/data/raw/eter/eter_export_all.csv',encoding='utf-8',sep=';')

In [ ]:
eter.shape
    

In [ ]:
# for c in eter.columns:
#     print(c)

In [ ]:
eter['Reference year'].value_counts()

In [ ]:
my_variables = ["English Institution Name",
                "Reference year",
                "Country Code",
                "Region of establishment (NUTS 3)",
                "Total students enrolled ISCED 5-7",
                "Students enrolled at ISCED 5-7 - Information and Communication Technologies",
                "Total graduates ISCED 5-7",
                "Graduates at ISCED 5-7 - Information and Communication Technologies",
                "Total students enrolled at ISCED 8",
                "Students enrolled at ISCED 8 - Information and Communication Technologies",
                "Total graduates at ISCED 8",
                "Graduates at ISCED 8 - Information and Communication Technologies"]

In [ ]:
eter_filtered = eter[my_variables]

In [ ]:
student_counts = [x for x in eter_filtered.columns if 'ISCED' in x] 

for x in student_counts:
    
    print(x)
    eter_filtered[x] = eter_filtered[x].replace(['m','x','a','c','s',
                                                 'xr','xc'],[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    
    eter_filtered[x] = [int(x.split(',')[0]) if pd.isnull(x)==False else np.nan for x in eter_filtered[x]]    
            

In [ ]:
country_counts = eter_filtered.groupby([
    "Country Code",
    "Reference year"])[student_counts].sum().reset_index(drop=False)

In [ ]:
country_pivoted = country_counts.pivot_table(index='Country Code',
                           columns='Reference year',
                           values='Graduates at ISCED 5-7 - Information and Communication Technologies',
                          aggfunc='sum')

In [ ]:
fig,ax = plt.subplots(figsize=(5,8))

sn.heatmap(country_pivoted.sort_values(2016,ascending=False).drop(2017,axis=1),
          cmap='Purples')

save_fig('fig_6_eter.pdf',material_outputs)